In [6]:
from helpers import League,scrape

# Team Score Structure
- Teams
    - Team
        - Teamname
            - Shortname
    - Score
        - Home Score
        - Away Score
    - Team
        - Teamname
            - Shortname

In [13]:
class Match:
    def __init__(self,span = None):
        self.home_team = "none"
        self.away_team = "none"
        self.home_score = 0
        self.away_score = 0
        self.date = None
        self.stadium = "none"
        if span != None:
            self.populate(span)
            
    def populate(self,span):
        home_team_span =  span.find_all('span', {'class':'team'})[0]
        self.home_team = home_team_span.find_all('span', {'class':'shortname'})[0].text
        score = span.find_all('span' , {'class':'score'})[0].text
        self.home_score = int(score.split('-')[0])
        self.away_score = int(score.split('-')[1])
        away_team_span =  span.find_all('span', {'class':'team'})[1]
        self.away_team = away_team_span.find_all('span', {'class':'shortname'})[0].text
        
    def display_match(self):
        display = ""
        display+=self.home_team
        display+=" "
        display+=str(self.home_score)
        display+="-"
        display+=str(self.away_score)
        display+=" "
        display+=self.away_team
        print(display)

In [8]:
class Season:
    def __init__(self,url = None):
        self.years = " "
        self.matches = []
        self.league_table = {}
        self.df={}
        self.teams = ""
        if url!= None:
            self.scrape(url)
            
    def populate_matches(self,spans):
        for x in range(len(spans)):
            self.matches.append(Match(spans[x]))
        self.matches = self.matches[::-1]
    def display_matches(self):
        print(self.years)
        for x in range(len(self.matches)):
            self.matches[x].display_match()            
    
    def scrape(self,url):
        page = scrape(url)
        spans = page.find_all('span', {'class' : 'teams'})
        self.populate_matches(spans)
        self.years = page.find_all('div',{'class':'current'})[1].text
            
    def generate_league_table(self):
        self.teams = teams = set([x.home_team for x in self.matches if type(x.home_team)==str])
        for team in teams:
            self.league_table[team]= [0, 0, 0, 0, 0, 0, 0,"","","","","",0]
        for match in self.matches:
            if match.home_score > match.away_score:
                self.league_table[match.home_team][0]+=1
                self.league_table[match.home_team][3]+=3
                self.league_table[match.away_team][2]+=1
                for x in range(11,7,-1):
                    self.league_table[match.home_team][x] = self.league_table[match.home_team][x-1]
                for x in range(11,7,-1):
                    self.league_table[match.away_team][x] = self.league_table[match.away_team][x-1]
                self.league_table[match.home_team][7] = "W"
                self.league_table[match.away_team][7] = "L"

            if match.away_score > match.home_score:
                self.league_table[match.away_team][0]+=1
                self.league_table[match.away_team][3]+=3
                self.league_table[match.home_team][2]+=1
                for x in range(11,7,-1):
                    self.league_table[match.away_team][x] = self.league_table[match.away_team][x-1]
                self.league_table[match.away_team][7] = "W"
                for x in range(11,7,-1):
                    self.league_table[match.home_team][x] = self.league_table[match.home_team][x-1]
                self.league_table[match.home_team][7] = "L"
            if match.away_score == match.home_score:
                self.league_table[match.home_team][1]+=1
                self.league_table[match.away_team][1]+=1
                self.league_table[match.home_team][3]+=1
                self.league_table[match.away_team][3]+=1
                for x in range(11,7,-1):
                    self.league_table[match.away_team][x] = self.league_table[match.away_team][x-1]
                self.league_table[match.away_team][7] = "D"
                for x in range(11,7,-1):
                    self.league_table[match.home_team][x] = self.league_table[match.home_team][x-1]
                self.league_table[match.home_team][7] = "D"

            self.league_table[match.away_team][4]+=match.away_score-match.home_score
            self.league_table[match.home_team][4]+=match.home_score-match.away_score
            self.league_table[match.home_team][5]+=match.home_score
            self.league_table[match.home_team][6]+=match.away_score
            self.league_table[match.away_team][5]+=match.away_score
            self.league_table[match.away_team][6]+=match.home_score
            self.league_table[match.away_team][-1]+=1
            self.league_table[match.home_team][-1]+=1


    def display_league_table(self):
        for key, value in sorted(self.league_table.items(), key=lambda e: e[1][3])[::-1]:
            print (key, value)
            
    def process_league_table(self):
        self.df = {
            'Team' : [team for team in self.teams]
        }
        self.df['Matches Played'] = [self.league_table[team][12] for team in self.df['Team']]
        self.df['Wins'] = [self.league_table[team][0] for team in self.df['Team']]
        self.df['Draws'] = [self.league_table[team][1] for team in self.df['Team']]
        self.df['Loss'] = [self.league_table[team][2] for team in self.df['Team']]
        self.df['Points'] = [self.league_table[team][3] for team in self.df['Team']]    
        self.df['GD'] = [self.league_table[team][4] for team in self.df['Team']]
        self.df['GF'] = [self.league_table[team][5] for team in self.df['Team']]
        self.df['GA'] = [self.league_table[team][6] for team in self.df['Team']]
        self.df['Form'] = [self.league_table[team][7:12] for team in self.teams]
        return self.df
    def display_next_league(self):
        print (self.df['Team'])
        print (self.df['Matches Played'])
        print (self.df['Form'])


        

In [9]:
season = Season("https://www.premierleague.com/results")

In [10]:
#season.populate_matches(spans)
#season.display_matches()
season.generate_league_table()
df = season.process_league_table()
season.display_next_league()



['Southampton', 'Sheffield Utd', 'Leicester', 'Burnley', 'Aston Villa', 'Spurs', 'West Ham', 'Man Utd', 'Man City', 'Leeds', 'Wolves', 'Crystal Palace', 'Brighton', 'Everton', 'Newcastle', 'Fulham', 'West Brom', 'Liverpool', 'Arsenal', 'Chelsea']
[17, 17, 17, 15, 15, 16, 17, 16, 15, 17, 17, 17, 17, 16, 16, 15, 17, 17, 17, 17]
[['W', 'D', 'D', 'L', 'D'], ['L', 'L', 'L', 'D', 'L'], ['W', 'D', 'D', 'W', 'L'], ['W', 'L', 'W', 'D', 'W'], ['L', 'D', 'W', 'W', 'D'], ['W', 'D', 'L', 'L', 'D'], ['W', 'D', 'D', 'L', 'D'], ['W', 'W', 'D', 'W', 'W'], ['W', 'W', 'W', 'D', 'D'], ['L', 'W', 'W', 'L', 'W'], ['D', 'L', 'D', 'L', 'W'], ['W', 'D', 'L', 'L', 'D'], ['D', 'L', 'D', 'D', 'D'], ['L', 'W', 'W', 'W', 'W'], ['L', 'D', 'L', 'D', 'L'], ['D', 'D', 'D', 'D', 'L'], ['L', 'L', 'D', 'L', 'D'], ['L', 'D', 'D', 'W', 'W'], ['W', 'W', 'W', 'L', 'D'], ['L', 'D', 'L', 'W', 'L']]


In [11]:
import pandas as pd
table = pd.DataFrame(df)

In [12]:
table.sort_values(by = ["Points","GD"],ascending=False).reset_index(drop=True)

,Team,Matches Played,Wins,Draws,Loss,Points,GD,GF,GA,Form
0,Liverpool,17,9,6,2,33,16,37,21,"[L, D, D, W, W]"
1,Man Utd,16,10,3,3,33,9,33,24,"[W, W, D, W, W]"
2,Leicester,17,10,2,5,32,10,31,21,"[W, D, D, W, L]"
3,Spurs,16,8,5,3,29,14,29,15,"[W, D, L, L, D]"
4,Man City,15,8,5,2,29,11,24,13,"[W, W, W, D, D]"
5,Southampton,17,8,5,4,29,7,26,19,"[W, D, D, L, D]"
6,Everton,16,9,2,5,29,6,26,20,"[L, W, W, W, W]"
7,Aston Villa,15,8,2,5,26,13,29,16,"[L, D, W, W, D]"
8,Chelsea,17,7,5,5,26,11,32,21,"[L, D, L, W, L]"
9,West Ham,17,7,5,5,26,3,24,21,"[W, D, D, L, D]"
